# Use test data from Ammon to test and valid Python iterative deconvolution

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

import obspy

In [ ]:
from seismic.receiver_fn.rf_deconvolution import iter_deconv_pulsetrain

In [ ]:
src_sac_z = "Ammon_test/lac_sp.z"
src_sac_r = "Ammon_test/lac_sp.r"
src_expected_rf = "Ammon_test/lac.i.eqr"
src_observed = "Ammon_test/observed"
src_predicted = "Ammon_test/predicted"
source = obspy.read(src_sac_z, format='sac')
response = obspy.read(src_sac_r, format='sac')
expected_rf = obspy.read(src_expected_rf, format='sac')
observed_radial = obspy.read(src_observed, format='sac')
predicted_radial = obspy.read(src_predicted, format='sac')

In [ ]:
# Run algorithm on Ammon input
time_shift = 5.0
rf_trace, _, _, predicted_response, fit = iter_deconv_pulsetrain(
    source[0], response[0], 200, gwidth=2.5, time_shift=time_shift)

In [ ]:
# Plot the results of expected vs computed
times = np.arange(len(observed_radial[0]))/observed_radial[0].stats.sampling_rate - time_shift
plt.figure(figsize=(12, 8))
plt.plot(times, observed_radial[0].data, alpha=0.8)
# plt.plot(times, expected_response, ":", alpha=0.8)
plt.plot(times, predicted_radial[0].data, alpha=0.8)
plt.plot(times, predicted_response.data, alpha=0.8)
plt.xlabel("Time (s)")
plt.ylabel("Radial amplitude")
plt.text(0.02, 0.02, "Prediction match to observation: {:.2f}%".format(fit), transform=plt.gca().transAxes)
plt.grid("#80808080", linestyle=':')
plt.legend(['Observed', 'Predicted observation (Ammon)', 'Predicted observation (Python port)'])
plt.title("Iterative deconv test, Radial component")
# plt.savefig('ammon_iterdeconvd_observation.png', dpi=300)
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(12, 8))
# Normalize so that sum of squares is 1.
sum_sq = np.sum(np.square(expected_rf[0].data))
plt.plot(times, expected_rf[0].data/np.sqrt(sum_sq), '-o', markersize=2, fillstyle='none', alpha=0.8)
sum_sq = np.sum(np.square(rf_trace.data))
plt.plot(times, rf_trace.data/np.sqrt(sum_sq), '-^', markersize=2, fillstyle='none', alpha=0.8)
plt.xlabel("Time (s)")
plt.ylabel("RF amplitude (arb. units)")
plt.grid("#80808080", linestyle=':')
plt.legend(['Ammon reference RF', 'Computed RF (Python port)'])
plt.title("Iterative deconv test, Receiver Function")
plt.show()
# plt.savefig('ammon_iterdeconvd_RF.png', dpi=300)
plt.close()